In [1]:
import numpy as np
import itertools
import sympy as sp
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Define the four functions

def identity(x):
    return x

def swap(x):
    return 1-x

def reset0(_):
    return 0

def reset1(_):
    return 1

base_strategies = [identity, swap, reset0, reset1]
all_strategies = list(itertools.product(base_strategies, repeat=2))

In [3]:
#generating possible deterministic P_XY|ZU values for all given u
def generate_top(u):
    f_u, g_u = all_strategies[u]
    #print(f"f_{u} = {f_u.__name__} \ng_{u} = {g_u.__name__}\n")

    def inner_entry(z):
        inner = np.zeros((2,2), dtype=int)
        x = f_u(z)
        y = g_u(x)
        inner[y, x] = 1

        return inner

    outer_level = np.array([inner_entry(0), inner_entry(1)])
    
    return outer_level

In [4]:
generate_top(0)

array([[[1, 0],
        [0, 0]],

       [[0, 0],
        [0, 1]]])

In [5]:
# declaring Gurobi variable W_i w/ i in card_u, and declaring v
m = gp.Model()
W = m.addVars(16, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="W")
v = m.addVar(lb=0, ub=1, vtype=GRB.CONTINUOUS, name="v")
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [6]:
# sample compatible distribution

prob_dist = {
    (0, 0, 0): 0.0, 
    (0, 0, 1): 0.03571428571428571, 
    (0, 1, 0): 0.07142857142857142, 
    (0, 1, 1): 0.10714285714285714, 
    (1, 0, 0): 0.14285714285714285, 
    (1, 0, 1): 0.17857142857142858, 
    (1, 1, 0): 0.21428571428571427, 
    (1, 1, 1): 0.25}

cardX, cardY, cardZ = 2, 2, 2

def P_Z(z):
    return sum([prob_dist[(z, X, Y)] for X in range(cardX) for Y in range(cardY)])
def P_ZXY(z, x, y):
    return prob_dist[(z, x, y)]
def P_XY_given_Z(x,y,z):
    return P_ZXY(z,x,y)/P_Z(z)


In [7]:
internal_gurobi_expr = sum([W[i]*generate_top(i) for i in range(16)])

internal_gurobi_expr $[z,x,y]$  
P_XY_giv_Z $(x,y,z)$

Found summed strategies matrices for all U! (internal_gurobi_exp)

## Gurobi Constraints

In [8]:
# Add the constraint to the model
m.addConstr(W.sum() == 1, "sum(W_i) == 1")
for z, x, y in itertools.product(range(2), repeat=3):
    m.addConstr(internal_gurobi_expr[z,x,y] == P_XY_given_Z(x,y,z), f"P({x},{y}|{z}) == {internal_gurobi_expr[z,x,y]} == {P_XY_given_Z(x,y,z)}")


# for cons in m.getConstrs():
#     print(cons)

# reset gurobi constraints
# m.remove(m.getConstrs())

m.update()

In [9]:
# maximum v?
# m.setObjective(v, GRB.MAXIMIZE)

obj = sum([W[i] for i in range(16)])
m.setObjective(obj, GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 9 rows, 17 columns and 48 nonzeros
Model fingerprint: 0xd7978c91
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 1e+00]
Presolve removed 9 rows and 17 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.000000000e+00


In [10]:
for i in range(16):
    print(f"W_{i} = {W[i].x}")

W_0 = 0.0
W_1 = 0.2272727272727273
W_2 = 0.0
W_3 = 0.10606060606060602
W_4 = 0.18181818181818182
W_5 = 0.16666666666666674
W_6 = 0.0
W_7 = 0.10606060606060602
W_8 = 0.0
W_9 = 0.0
W_10 = 0.0
W_11 = 0.0
W_12 = 0.21212121212121215
W_13 = 0.0
W_14 = 0.0
W_15 = 0.0
